In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
# 신경망 모델 구조 파일
cfg_file = './opencv_data/yolo/yolov3.cfg'

# 가중치 파일
weights_file = './opencv_data/yolo/yolov3.weights'

# 인식 가능한 사물의 이름(파일)
class_file = './opencv_data/yolo/coco.names'

In [3]:
# yolo 모델을 복원.
# cv2.dnn.readNet(신경망 데이터, 가중치 데이터)
net = cv2.dnn.readNet(weights_file, cfg_file)
net

< cv2.dnn.Net 000001D6391974B0>

In [4]:
# 물체 종류를 리스트로 추출.
# 이미지속에 있는 물체들.. ex) 사람, 자동차, 나무 등..

with open(class_file, 'rt') as fp :
    classes = fp.readlines()
    # \n 제거
    classes = [line.strip() for line in classes]
    
classes  

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [5]:
# 각 물체를 의미하는(가리키는) 색상을 랜덤하게 생성.
# 0 ~ 255 사이 
# r, g, b
colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors

array([[101.94796608, 235.70840187,  59.12755141],
       [143.22070115, 245.33652176,  61.86591291],
       [222.01429184, 195.12178967,   5.03792529],
       [ 63.02576773,  69.30901786,  20.29034193],
       [159.8322874 ,  51.93713859, 117.12739412],
       [  5.78058367,  16.26709533, 245.75208517],
       [141.58649718, 251.31314898, 113.32560575],
       [163.18775742, 105.41093264, 195.88889444],
       [ 62.32582653, 100.74530822, 127.00476294],
       [202.36897259, 178.01973735, 184.82526816],
       [176.70097887, 239.72039562, 144.88027085],
       [ 18.20968658,  45.11383301,  70.07304014],
       [ 67.18460265,  52.97645867,  78.57847074],
       [ 28.70603412,  64.43976224, 233.9406463 ],
       [ 23.30409159, 175.46496336, 174.18164183],
       [173.17209822,  84.47690389,  32.57344417],
       [157.47435184, 111.46035087, 137.04720376],
       [108.64367479, 196.642512  ,  17.03235471],
       [ 11.30025059,  97.9862519 ,  59.43663759],
       [ 19.80489162, 123.45038

In [6]:
# 사용할 영상을 가져온다.
cap = cv2.VideoCapture('./opencv_data/video/yolo_01.mp4')

In [7]:
# 엔터키를 누를때 까지 반복.
while cv2.waitKey(33) < 0 :
    # 현재 프레임을 가져옴.
    ret, img = cap.read()
    # 더이상 가져올 것이 없다면 (끝까지 갔을 경우) 종료
    if img is None :
        break
#   ------------↓ yolo 사진 내용과 동일 ---------------      
    # 이미지 세로, 가로, 채널
    height, width, channel = img.shape
    
    # 이미지 크기 조정. cv2.resize(원본데이터, (세로,가로))
    # 16*16 부터 16 배수로 설정.
    a1 = cv2.resize(img, (416,416))
    
    # 2진 데이터로 변환.
    # 0.00392(기준) - 임계값. 이 값보다 크면 1, 작으면 0으로 변환.
    blob = cv2.dnn.blobFromImage(a1, 0.00392, (416, 416), (0, 0, 0))
    
    # 예측 전 신경망 구조를 살핌.
    # 모델의 정보를 확인. 은닉층들의 이름을 가져옴.
    # 이 부분은 생략 가능
    
    # 출력층 이름.
    output_layers = net.getUnconnectedOutLayersNames()
    # 결론 : 신경망이 3개.
    
    # 이미지 데이터를 모델에 셋팅.
    net.setInput(blob)
    
    # 물체를 검출. 출력층이 3개이기때문에 ndarray도 3개 
    outs = net.forward(output_layers)


    # 예측된 결과를 담을 리스트
    class_id_list = []
    # 예측 정확도
    confidence_list = []
    # 인지된 사물의 영역
    box_list = []

    # 확률의 임계값
    # 이 확률 이상인 것만 결과에 담는다.
    # 0.1 ~ 2 정도로 잡아도 충분함.
    confidence_limit = 0.5

    for out in outs :
        # 현재의 출력층이 검출한 물체의 수 만큼 반복.
        for detection in out :
            # 원핫 인코딩되어 있는 확률값만 가져온다.
            score_list = detection[5:]
            # 확률이 가장 높은 물체 값을 가져온다.
            class_id = np.argmax(score_list)
            # 확률을 가져온다.
            confidence = score_list[class_id]
    #         print(classes[class_id] , score_list[class_id])

            # 확률이 혹률 임계값 이상인 것만 처리.
            if confidence >= confidence_limit:
     
                # 물체의 좌표를 계산.
                # 예측결과에서 앞 4개는 중심점 x 비율, 중심점 y비율, 가로비율, 세로비율
                # 원본 이미지로 환산
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)

                # 가로길이
                w = int(detection[2] * width)
                # 세로길이
                h = int(detection[3] * height)
                # 좌측 상단 x
                x = int(center_x - w / 2)
                # 좌측 상단 y
                y = int(center_y - h / 2)        

                # 담는다.
                box_list.append([x, y, w, h])
                confidence_list.append(float(confidence))
                class_id_list.append(class_id)

    # IoU : 두 면적이 중첩되는 영역의 넓이를 두 면적의 함친 총 면적으로 나눈 값
    # 두 면적이 얼마나 중첩되어 있는지 평가할 수 있는 지표
    # 0 ~ 1 사이가 나오며 값이 클수록 중첩된 부분이 많다고 평가.

    # NMS : IoU 방식으로 면적을 평가하고 중첩이 많이 되었다고 판단되는 영역을 제거하는 방식.
    # IoU 방식으로 평가하여 중첩이 많이 되었다고 판단되는 영역을 모두 아우를수 있는 영역을 만들어 냄.
    # cv2.dnn.NMSBoxes( 사물영역, 예측정확도, 확률임계값, 확인하지않고 버릴 영역의 확률값(확률임계값보다 낮게))

    # 결과는 인식한 사물의 index 번호
    indexes = cv2.dnn.NMSBoxes(box_list, confidence_list, confidence_limit, 0.4)
    
    # 검출된 물체의 수 만큼 반복.

    for i in range(len(box_list)) :
        # NMS를 통해 추출한 영역만 표시.
        if i in indexes : 

            # 좌표값을 추출.
            x, y, w, h = box_list[i]
            # 이름.
            label = str(classes[class_id_list[i]])
            # 색상.
            color = colors[class_id_list[i]]

            # 네모를 그린다.
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            # 이름을 표시. cv2.FONT_HERSHEY_PLAIN(폰트), 크기, 색상, 두께
            cv2.putText(img, label, (x, y-30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

    cv2.imshow('Object Detection', img)
    
cap.release()
cv2.destroyAllWindows()